In [ ]:
!pip install openai python-aiconfig

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.6/381.6 kB 24.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.2/133.2 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 70.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.9/267.9 kB 27.2 MB/s

In [ ]:
import openai
import aiconfig
from google.colab import userdata
openai.api_key = userdata.get('OPENAI_API_KEY')


/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_parsers" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [ ]:
spec = {
  "name": "NYC Trip Planner",
  "description": "Intrepid explorer with ChatGPT and AIConfig",
  "schema_version": "latest",
  "metadata": {
    "models": {
      "gpt-3.5-turbo": {
        "model": "gpt-3.5-turbo",
        "top_p": 1,
        "temperature": 1
      },
      "gpt-4": {
        "model": "gpt-4",
        "max_tokens": 3000,
        "system_prompt": "You are an expert travel coordinator with exquisite taste."
      }
    },
    "default_model": "gpt-3.5-turbo"
  },
  "prompts": [
    {
      "name": "get_activities",
      "input": "Tell me 10 unique luxury activities in NYC."
    },
    {
      "name": "gen_itinerary",
      "input": "Generate a ranked list ordered by {{order_by}} for these activities: {{get_activities.output}}. Include estimated cost.",
      "metadata": {
        "model": "gpt-4",
        "parameters": {
          "order_by": "activity expense, from most to least"
        }
      }
    }
  ]
}

In [ ]:
from aiconfig import AIConfigRuntime, InferenceOptions

# Load the aiconfig.
config = AIConfigRuntime.create(**spec)

# Set the inference options.

# Run a single prompt
response = await config.run("get_activities")


INFO:my-logger:Callback called. event
: name='on_run_start' file='aiconfig.Config' data={'prompt_name': 'get_activities', 'params': None, 'options': None, 'kwargs': {}} ts_ns=1700185913158115460
INFO:my-logger:Callback called. event
: name='on_run_start' file='aiconfig.default_parsers.openai' data={'prompt': Prompt(name='get_activities', input='Tell me 10 unique luxury activities in NYC.', metadata=None, outputs=[]), 'options': None, 'parameters': {}} ts_ns=1700185913158115460
INFO:my-logger:Callback called. event
: name='on_deserialize_start' file='aiconfig.default_parsers.openai' data={'prompt': Prompt(name='get_activities', input='Tell me 10 unique luxury activities in NYC.', metadata=None, outputs=[]), 'params': {}} ts_ns=1700185913158115460
INFO:my-logger:Callback called. event
: name='on_deserialize_complete' file='aiconfig.default_parsers.openai' data={'output': {'top_p': 1, 'model': 'gpt-3.5-turbo', 'temperature': 1, 'messages': [{'content': 'Tell me 10 unique luxury activities

In [ ]:
response = await config.run("gen_itinerary")

INFO:my-logger:Callback called. event
: name='on_run_start' file='aiconfig.Config' data={'prompt_name': 'gen_itinerary', 'params': None, 'options': None, 'kwargs': {}} ts_ns=1700185913158115460
INFO:my-logger:Callback called. event
: name='on_run_start' file='aiconfig.default_parsers.openai' data={'prompt': Prompt(name='gen_itinerary', input='Generate a ranked list ordered by {{order_by}} for these activities: {{get_activities.output}}. Include estimated cost.', metadata=PromptMetadata(model='gpt-4', tags=None, parameters={'order_by': 'activity expense, from most to least'}), outputs=[]), 'options': None, 'parameters': {}} ts_ns=1700185913158115460
INFO:my-logger:Callback called. event
: name='on_deserialize_start' file='aiconfig.default_parsers.openai' data={'prompt': Prompt(name='gen_itinerary', input='Generate a ranked list ordered by {{order_by}} for these activities: {{get_activities.output}}. Include estimated cost.', metadata=PromptMetadata(model='gpt-4', tags=None, parameters={

In [ ]:
from pprint import pprint

In [ ]:
pprint(response[0].data)

{'content': '1. Private Yacht Charter: Cruise along the Hudson or East River '
            'on a luxury yacht with a dedicated crew. Estimated cost: '
            '$20,000-25,000 per day.\n'
            '   \n'
            "2. Private Helicopter Tour: Take a bird's-eye view of New York "
            "City's iconic landmarks. Estimated cost: $1,700-2,000 per hour.\n"
            '\n'
            '3. Exclusive Dining Experience: Enjoy a personalized dining '
            'experience with a famous chef in a private room with panoramic '
            'city views. Estimated cost: $1,000-1,500 per person.\n'
            '\n'
            '4. Supercar Experience: Drive a luxury supercar, like a '
            "Lamborghini or Ferrari, through NYC's neighborhoods. Estimated "
            "cost: $900-1,200 for a day's rental. \n"
            '\n'
            '5. VIP Broadway Experience: Get backstage tours, meet Broadway '
            'cast members, and gain access to premium seats for popular shows

## Function Calling

In [ ]:
from pydantic import create_model
from typing import Callable, List
import inspect, json
from inspect import Parameter
def sums(a:int, b:int=1):
    "Adds a + b"
    return a + b
def schema(f):
    kw = {n:(o.annotation, ... if o.default==Parameter.empty else o.default)
          for n,o in inspect.signature(f).parameters.items()}
    s = create_model(f'Input for `{f.__name__}`', **kw).schema()
    return dict(name=f.__name__, description=f.__doc__, parameters=s)

In [ ]:
def build_function_call_spec(functions: List[Callable]):
    spec = json.loads("""
        {
            "name": "function-call-demo",
            "description": "this is a demo AIConfig to show function calling using OpenAI",
            "schema_version": "latest",
            "metadata": {},
            "prompts": [
                {
                "name": "recommendBook",
                "input": {
                    "role": "user",
                    "content": "I really enjoyed reading {{book}}, could you recommend me a book that is similar and tell me why?"
                },
                "metadata": {
                    "model": {
                        "name": "gpt-3.5-turbo",
                        "settings": {
                            "model": "gpt-3.5-turbo",
                            "functions": [
                            ],
                            "system_prompt": {
                            "role": "system",
                            "content": "Please use our book database, which you can access using functions to answer the following questions."
                            }
                        }
                    },
                    "parameters": {},
                    "remember_chat_context": true
                }
                }
            ]
        }
    """)
    spec["prompts"][0]["metadata"]["model"]["settings"]["functions"] = [schema(f) for f in functions]
    return spec



In [ ]:
import copy
import json
import uuid
from aiconfig import AIConfigRuntime
from aiconfig import Prompt
from aiconfig import PromptInput
from aiconfig.model_parser import InferenceOptions
from pprint import pprint


async def function_calling(function_list):
    function_call_spec = build_function_call_spec(function_list)
    config = AIConfigRuntime.create(**function_call_spec)

    params = {
        "book": "Where the Crawdads Sing",
    }

    completionParams = await config.resolve("recommendBook", params)

    print("completionParams=", end=" ")
    pprint(completionParams)

    messages = completionParams["messages"]
    print(messages[0])
    print(messages[1])
    print()

    promptToRun = "recommendBook"

    inference_options = InferenceOptions(stream=True)

    while True:
        model_output = await config.run(promptToRun, params, inference_options)

        output = model_output[0] if isinstance(model_output, list) else model_output

        if output.output_type == "error":
            print(f"Error during inference: {output.ename}: {output.evalue}")
            return config

        message = output.data

        # If there is no function call, we're done and can exit this loop
        if not message.get("function_call", None):
            return config

        # If there is a function call, we generate a new message with the role 'function'.

        result = await callFunction(message.get("function_call"))

        new_message = {
            "role": "function",
            "name": message["function_call"]["name"],
            "content": json.dumps(result),
        }

        promptToRun = f"functionCallResult-{uuid.uuid4()}"

        existing_prompt = config.get_prompt("recommendBook")

        new_prompt = copy.deepcopy(existing_prompt)
        new_prompt.name = promptToRun
        new_prompt.input = PromptInput(**new_message)
        new_prompt.outputs = []

        config.add_prompt(new_prompt.name, new_prompt)

        print(f"{new_message}\n")


db = [
    {
        "id": "a1",
        "name": "To Kill a Mockingbird",
        "genre": "historical",
        "description": 'Compassionate, dramatic, and deeply moving, "To Kill A Mockingbird" takes readers to the roots of human behavior - to innocence and experience, kindness and cruelty, love and hatred, humor and pathos. Now with over 18 million copies in print and translated into forty languages, this regional story by a young Alabama woman claims universal appeal. Harper Lee always considered her book to be a simple love story. Today it is regarded as a masterpiece of American literature.',
    },
    {
        "id": "a2",
        "name": "All the Light We Cannot See",
        "genre": "historical",
        "description": "In a mining town in Germany, Werner Pfennig, an orphan, grows up with his younger sister, enchanted by a crude radio they find that brings them news and stories from places they have never seen or imagined. Werner becomes an expert at building and fixing these crucial new instruments and is enlisted to use his talent to track down the resistance. Deftly interweaving the lives of Marie-Laure and Werner, Doerr illuminates the ways, against all odds, people try to be good to one another.",
    },
    {
        "id": "a3",
        "name": "Where the Crawdads Sing",
        "genre": "historical",
        "description": """For years, rumors of the “Marsh Girl” haunted Barkley Cove, a quiet fishing village. Kya Clark is barefoot and wild; unfit for polite society. So in late 1969, when the popular Chase Andrews is found dead, locals immediately suspect her.

But Kya is not what they say. A born naturalist with just one day of school, she takes life's lessons from the land, learning the real ways of the world from the dishonest signals of fireflies. But while she has the skills to live in solitude forever, the time comes when she yearns to be touched and loved. Drawn to two young men from town, who are each intrigued by her wild beauty, Kya opens herself to a new and startling world—until the unthinkable happens.""",
    },
]


async def list_items(genre: str):
    """list queries books by genre, and returns a list of names of books"""
    # filter
    items = [item for item in db if item["genre"] == genre]
    # map
    return [{"name": item["name"], "id": item["id"]} for item in items]


async def search(name: str):
    """search queries books by their name and returns a list of book names and their ids"""
    # filter
    items = [item for item in db if name in item["name"]]
    # map
    return [{"name": item["name"], "id": item["id"]} for item in items]


async def get(id: str):
    """get returns a book's detailed information based on the id of the book. Note that this does not accept names, and only IDs, which you can get by using search."""
    return [item for item in db if item["id"] == id][0] or None


async def callFunction(function_call):
    args = function_call.get("arguments", None)
    args = json.loads(args) if args else None

    if not args:
        raise Exception("No arguments found")

    match function_call.get("name"):
        case "list_items":
            return await list_items(args["genre"])
        case "search":
            return await search(args["name"])
        case "get":
            return await get(args["id"])



In [ ]:
book_search = await function_calling([list_items, search, get])

<ipython-input-39-d40b6c76d851>:11: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  s = create_model(f'Input for `{f.__name__}`', **kw).schema()
INFO:my-logger:Callback called. event
: name='on_resolve_start' file='/usr/local/lib/python3.10/dist-packages/aiconfig/Config.py' data={'prompt_name': 'recommendBook', 'params': {'book': 'Where the Crawdads Sing'}} ts_ns=1700185913158115460
INFO:my-logger:Callback called. event
: name='on_deserialize_start' file='aiconfig.default_parsers.openai' data={'prompt': Prompt(name='recommendBook', input=PromptInput(data=None, role='user', content='I really enjoyed reading {{book}}, could you recommend me a book that is similar and tell me why?'), metadata=PromptMetadata(model=ModelMetadata(name='gpt-3.5-turbo', settings={'model': 'gpt-3.5-turbo', 'functions': [{'name': 'list_i

completionParams= {'functions': [{'description': 'list queries books by genre, and returns a '
                               'list of names of books',
                'name': 'list_items',
                'parameters': {'properties': {'genre': {'title': 'Genre',
                                                        'type': 'string'}},
                               'required': ['genre'],
                               'title': 'Input for `list_items`',
                               'type': 'object'}},
               {'description': 'search queries books by their name and returns '
                               'a list of book names and their ids',
                'name': 'search',
                'parameters': {'properties': {'name': {'title': 'Name',
                                                       'type': 'string'}},
                               'required': ['name'],
                               'title': 'Input for `search`',
                               'type': 'obj

INFO:my-logger:Callback called. event
: name='on_run_complete' file='aiconfig.default_parsers.openai' data={'result': [ExecuteResult(output_type='execute_result', execution_count=0, data={'function_call': {'arguments': '{\n"name": "Where the Crawdads Sing"\n}', 'name': 'search'}, 'role': 'assistant'}, mime_type=None, metadata={'finish_reason': 'function_call'})]} ts_ns=1700185913158115460
INFO:my-logger:Callback called. event
: name='on_run_complete' file='aiconfig.Config' data={'result': [ExecuteResult(output_type='execute_result', execution_count=0, data={'function_call': {'arguments': '{\n"name": "Where the Crawdads Sing"\n}', 'name': 'search'}, 'role': 'assistant'}, mime_type=None, metadata={'finish_reason': 'function_call'})]} ts_ns=1700185913158115460
INFO:my-logger:Callback called. event
: name='on_run_start' file='aiconfig.Config' data={'prompt_name': 'functionCallResult-ed6463b4-93ba-4446-bf26-2ff771be0b99', 'params': {'book': 'Where the Crawdads Sing'}, 'options': <aiconfig.m

{'role': 'function', 'name': 'search', 'content': '[{"name": "Where the Crawdads Sing", "id": "a3"}]'}



INFO:my-logger:Callback called. event
: name='on_run_complete' file='aiconfig.default_parsers.openai' data={'result': [ExecuteResult(output_type='execute_result', execution_count=0, data={'function_call': {'arguments': '{\n"id": "a3"\n}', 'name': 'get'}, 'role': 'assistant'}, mime_type=None, metadata={'finish_reason': 'function_call'})]} ts_ns=1700185913158115460
INFO:my-logger:Callback called. event
: name='on_run_complete' file='aiconfig.Config' data={'result': [ExecuteResult(output_type='execute_result', execution_count=0, data={'function_call': {'arguments': '{\n"id": "a3"\n}', 'name': 'get'}, 'role': 'assistant'}, mime_type=None, metadata={'finish_reason': 'function_call'})]} ts_ns=1700185913158115460
INFO:my-logger:Callback called. event
: name='on_run_start' file='aiconfig.Config' data={'prompt_name': 'functionCallResult-cafb8ce9-78bb-4d24-8aa5-833c8f002225', 'params': {'book': 'Where the Crawdads Sing'}, 'options': <aiconfig.model_parser.InferenceOptions object at 0x7a2f97cfa92

{'role': 'function', 'name': 'get', 'content': '{"id": "a3", "name": "Where the Crawdads Sing", "genre": "historical", "description": "For years, rumors of the \\u201cMarsh Girl\\u201d haunted Barkley Cove, a quiet fishing village. Kya Clark is barefoot and wild; unfit for polite society. So in late 1969, when the popular Chase Andrews is found dead, locals immediately suspect her.\\n\\nBut Kya is not what they say. A born naturalist with just one day of school, she takes life\'s lessons from the land, learning the real ways of the world from the dishonest signals of fireflies. But while she has the skills to live in solitude forever, the time comes when she yearns to be touched and loved. Drawn to two young men from town, who are each intrigued by her wild beauty, Kya opens herself to a new and startling world\\u2014until the unthinkable happens."}'}



INFO:my-logger:Callback called. event
: name='on_run_complete' file='aiconfig.default_parsers.openai' data={'result': [ExecuteResult(output_type='execute_result', execution_count=0, data={'function_call': {'arguments': '{\n"genre": "historical"\n}', 'name': 'list_items'}, 'role': 'assistant'}, mime_type=None, metadata={'finish_reason': 'function_call'})]} ts_ns=1700185913158115460
INFO:my-logger:Callback called. event
: name='on_run_complete' file='aiconfig.Config' data={'result': [ExecuteResult(output_type='execute_result', execution_count=0, data={'function_call': {'arguments': '{\n"genre": "historical"\n}', 'name': 'list_items'}, 'role': 'assistant'}, mime_type=None, metadata={'finish_reason': 'function_call'})]} ts_ns=1700185913158115460
INFO:my-logger:Callback called. event
: name='on_run_start' file='aiconfig.Config' data={'prompt_name': 'functionCallResult-524bb68e-56cb-4e26-8f6b-92154f5cb4b2', 'params': {'book': 'Where the Crawdads Sing'}, 'options': <aiconfig.model_parser.Infe

{'role': 'function', 'name': 'list_items', 'content': '[{"name": "To Kill a Mockingbird", "id": "a1"}, {"name": "All the Light We Cannot See", "id": "a2"}, {"name": "Where the Crawdads Sing", "id": "a3"}]'}

I would recommend the book "All the Light We Cannot See" as a similar book to "Where the Crawdads Sing". 

"All the Light We Cannot See" is also a historical novel, like "Where the Crawdads Sing". It tells the story of a blind French girl and a young German boy whose paths collide in occupied France during World War II. The book explores themes of resilience, bravery, and the power of human connection amidst the backdrop of war. 

Both "Where the Crawdads Sing" and "All the Light We Cannot See" have captivating storytelling, richly drawn characters, and a strong sense of place. They both transport the reader to a different time and immerse them in the lives of the characters. If you enjoyed the atmospheric and immersive nature of "Where the Crawdads Sing", you are likely to enjoy "

INFO:my-logger:Callback called. event
: name='on_run_complete' file='aiconfig.default_parsers.openai' data={'result': [ExecuteResult(output_type='execute_result', execution_count=0, data={'content': 'I would recommend the book "All the Light We Cannot See" as a similar book to "Where the Crawdads Sing". \n\n"All the Light We Cannot See" is also a historical novel, like "Where the Crawdads Sing". It tells the story of a blind French girl and a young German boy whose paths collide in occupied France during World War II. The book explores themes of resilience, bravery, and the power of human connection amidst the backdrop of war. \n\nBoth "Where the Crawdads Sing" and "All the Light We Cannot See" have captivating storytelling, richly drawn characters, and a strong sense of place. They both transport the reader to a different time and immerse them in the lives of the characters. If you enjoyed the atmospheric and immersive nature of "Where the Crawdads Sing", you are likely to enjoy "All 

In [ ]:
book_search.save("book_search.json")

FileNotFoundError: ignored